# KALMAN FOLDING IN CLOJURE

Some sample data:

In [1]:
(def zs [-0.178654, 0.828305, 0.0592247, -0.0121089, -1.48014, 
         -0.315044, -0.324796, -0.676357, 0.16301, -0.858164])

#'user/zs

TODO: generate new random data.

TODO: make it work over core.async or Rx or whatever.

TODO: incanter integration

## RUNNING COUNT

The traditional and obvious way:

In [2]:
(reduce (fn [c z] (inc c)) 0 zs)

10

A thread-safe way:

In [3]:
(let [initial-count 0
      running-count (atom initial-count)]
    (reduce (fn [c z]
                (swap! running-count inc)
                @running-count) ; ~~> new value for c
            initial-count
            zs))

10

A thread-safe way, without `reduce`:

In [4]:
(let [initial-count 0
      running-count (atom initial-count)]
    (dorun
        (map (fn [z]
                 (swap! running-count inc))
             zs))
    @running-count)

10

## RUNNING MEAN

In [74]:
(let [initial-count 0
      initial-mean  0
      initial-stats {:count initial-count
                     :mean  initial-mean}
      running-stats (atom initial-stats)]
    (dorun
        (map (fn [z]
                 (let [{x :mean, n :count} @running-stats
                       n+1 (inc n)]
                     (swap! running-stats conj
                            [:count n+1]
                            [:mean (let [K (/ 1.0 n+1)]
                                           (+ x (* K (- z x))))])))
             zs))
    @running-stats)

{:count 10, :mean -0.27947242}

We can refactor this to provide the running mean at every step. This is like a `scan` in Haskell or a `FoldList` in Wolfram. It's also more hopitable to reapplication on an async stream.

In [73]:
(defn make-running-stats-mapper []
    (let [initial-count 0
          initial-mean  0
          initial-stats {:count initial-count :mean  initial-mean}
          running-stats (atom initial-stats)]
        (fn [z]
            (let [{x :mean, n :count} @running-stats
                   n+1 (inc n)]
                (swap! running-stats conj
                       [:count n+1]
                       [:mean (let [K (/ 1.0 n+1)]
                                  (+ x (* K (- z x))))]))
            @running-stats)))
(clojure.pprint/pprint (map (make-running-stats-mapper) zs))

({:count 1, :mean -0.178654}
 {:count 2, :mean 0.3248255}
 {:count 3, :mean 0.2362919}
 {:count 4, :mean 0.1741917}
 {:count 5, :mean -0.15667464000000003}
 {:count 6, :mean -0.18306953333333337}
 {:count 7, :mean -0.20331617142857145}
 {:count 8, :mean -0.262446275}
 {:count 9, :mean -0.21517335555555556}
 {:count 10, :mean -0.27947242})


### NUMERICAL CHECK

In [76]:
(/ (reduce + zs) (count zs))

-0.27947242

## CORE.ASYNC

In [77]:
(require '[clojure.core.async :refer [onto-chan sliding-buffer 
                                      dropping-buffer buffer
                                      <!! <! >! >!! go chan 
                                      close! thread 
                                      alts! alts!! timeout]])

We can write on the asynch thread and read on the UI thread:

In [78]:
(let [c (chan)]
    (go (>! c 42))
    (println (<!! c))
    (close! c))

42


We can read on the async thread and write on the UI thread:

In [79]:
(let [c (chan)]
    (go (println (<! c)))
    (>!! c 42)
    (close! c))

42


In all cases, we must do any blocking call without timeout on the UI thread last. The following will hang:

In [80]:
#_(let [c (chan)]
    (>!! c 42)
    (go (println (<! c)))
    (close! c))

We're won't block if we add a timeout, but we don't know how to write to a timeout channel:

In [80]:
(let [c (chan)]
    (>!! (alts!! [c (timeout 500)]) 42)
    (go (println (<! c)))
    (close! c))

IllegalArgumentException No implementation of method: :put! of protocol: #'clojure.core.async.impl.protocols/WritePort found for class: clojure.lang.PersistentVector  clojure.core/-cache-protocol-fn (core_deftype.clj:568)


class java.lang.IllegalArgumentException: 

We can't read from a `timeout` channel either, but at least we won't hang. Here, we do the blocking read too early on the UI thread:

In [80]:
(let [c (chan)]
    (println (<!! (alts!! [c (timeout 500)])))
    (go (>! c 42))
    (close! c))

IllegalArgumentException No implementation of method: :take! of protocol: #'clojure.core.async.impl.protocols/ReadPort found for class: clojure.lang.PersistentVector  clojure.core/-cache-protocol-fn (core_deftype.clj:568)


class java.lang.IllegalArgumentException: 

The following illustrates putting data in a blocking channel at random times and reading some of them on the UI thread. It will leave values in the channel and thus leak the channel according to the documentation for `close!` here https://clojure.github.io/core.async/api-index.html#C.

In [81]:
(def echo-chan (chan))

(doseq   [z zs] (go (Thread/sleep (rand 100)) (>! echo-chan z)))
(dotimes [_ 3] (println (<!! echo-chan)))

(close! echo-chan)

0.828305
-0.315044
-0.324796


We can chain channels. Reading from `echo-chan` may hang the UI thread because the UI thread races the internal thread that reads `echo-chan`, but the timeout trick works here as above. This will also leak channels.

In [81]:
(def echo-chan (chan))
(def repl-chan (chan))

(dotimes [_ 10] (go (>! repl-chan (<! echo-chan))))
(doseq   [z zs] (go (Thread/sleep (rand 100)) (>! echo-chan z)))
(dotimes [_ 3] (println (<!! (alts!! [echo-chan (timeout 500)]))))

(close! echo-chan)
(close! repl-chan)

IllegalArgumentException No implementation of method: :take! of protocol: #'clojure.core.async.impl.protocols/ReadPort found for class: clojure.lang.PersistentVector  clojure.core/-cache-protocol-fn (core_deftype.clj:568)


class java.lang.IllegalArgumentException: 

`println` on a `go` process works if we wait long enough. This, of course, is bad practice or "code smell."

In [86]:
(def echo-chan (chan))

(doseq   [z zs] (go (Thread/sleep (rand 100)) (>! echo-chan z)))
(dotimes [_ 3]  (go (println (<! echo-chan))))

(Thread/sleep 500) ; no visible output if you remove this line.
(close! echo-chan)

-0.0121089
-1.48014
0.16301


### ASYNC RUNNING MEAN

We want our `running-stats` function called with the data delivered at random times and in random order. A transducer lets us collect items off the buffer. The size of the buffer does not matter.

In [90]:
(def echo-chan (chan (buffer 1) (map (make-running-stats-mapper))))
(doseq [z zs] (go (Thread/sleep (rand 100)) (>! echo-chan z)))
(dotimes [_ 10] (println (<!! echo-chan)))
(close! echo-chan)

{:count 1, :mean -0.324796}
{:count 2, :mean -0.251725}
{:count 3, :mean -0.6611966666666667}
{:count 4, :mean -0.49892472499999996}
{:count 5, :mean -0.46214858}
{:count 6, :mean -0.52815115}
{:count 7, :mean -0.5493234142857143}
{:count 8, :mean -0.47325490000000003}
{:count 9, :mean -0.3286371333333334}
{:count 10, :mean -0.27947242000000005}


## RUNNING STDDEV